In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Getting lemmas and metadata together****

In [2]:
%%time
# open file in read mode
with open('/kaggle/input/utf8tokenizedspeeches/TokenizedSpeeches_utf-8.csv', 'r') as read_obj:
#with open('TokenizedSpeeches_utf-8.csv', 'r') as read_obj:

    lemmasList = []
    
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader: 
        lemmasList.append(row)
    print(len(lemmasList), 'Rows read')

In [3]:
%%time
#create dataframe from the lemmas extracted from csv
dictOfLemmas = {'Lemmas': lemmasList}
lemmasDf = pd.DataFrame(dictOfLemmas)
lemmasDf

In [4]:
with open('/kaggle/input/hansard-speeches-lemmatized/hansard-speeches-post2010.pkl', 'rb') as f:
    df = pickle.load(f)

In [5]:
#since index was missing values and didn't match with the lemmasDf index
df = df.reset_index(drop=True)
df = df.join(lemmasDf)
df['Lemmas']

# Dividing and training corpus before and after the Brexit referendum

In [6]:
# Split data based on the Brexit referendum event before and after period
eventDate = '2016-06-23 23:59:59'
df_t1 = df[df['date']<= eventDate]
df_t2 = df[df['date']> eventDate]

# Choosing intersecting vocabulary and Aligning models 

In [7]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        if(len(m.wv.key_to_index)==135):
            print('Common vocab is', common_vocab)
        
    return (m1,m2)

In [8]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
        
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

# Splitting speeches by MPs and time

In [9]:
'''%%time
# Lets only consider the MPs who have speeches in both T1 & T2 time periods, intersection between two DFs 

t1List = pd.unique(df_t1['mnis_id']).tolist()
t2List = pd.unique(df_t2['mnis_id']).tolist()

intersectedList = list(set(t1List).intersection(t2List))

len(intersectedList)
# 651 MPs to be considered

# Now modify original df_t1 and df_t2 DFs to only contain speeches from the common MPs 

df_t1 = df_t1[df_t1['mnis_id'].isin(intersectedList)]
df_t2 = df_t2[df_t2['mnis_id'].isin(intersectedList)]
df_t2['Lemmas']'''

In [10]:

t1List = pd.unique(df_t1['mnis_id']).tolist()
t2List = pd.unique(df_t2['mnis_id']).tolist()
totalMps = set(t1List+t2List)
intersectedList = list(set(t1List).intersection(t2List))
print(len(totalMps), len(intersectedList))

In [11]:
%%time
# Original code to divide by MP and time
# This creates DFs with vocabs of each MP split per speech

'''dictSpeechesByMp = {}

for mpId in intersectedList:
    for dfTime in ['df_t1','df_t2']:
        dfName = dfTime + '_'+ mpId
        if(dfTime == 'df_t1'):
            dictSpeechesByMp[dfName]=df_t1[df_t1['mnis_id']==mpId]
        elif (dfTime == 'df_t2'):
            dictSpeechesByMp[dfName]=df_t2[df_t2['mnis_id']==mpId]
            '''

In [12]:
%%time
# New code for dividing corpus by time and MPs 

 #   -x  -x  -x  Integrates all lemmas to map per MP - one vocab per MP per time period (eventually two vocabs per MP across T1 & T2)  -x  -x  -x  -x  -x

dictSpeechesByMp = {}

for mpId in totalMps:

    for dfTime in ['df_t1','df_t2']:

        tempDf = pd.DataFrame()
        tempList = []
        Lemmas =[]
        dfName = dfTime + '_'+ mpId

        if(dfTime == 'df_t1'):
            tempDf = df_t1[df_t1['mnis_id']==mpId]
            
        elif (dfTime == 'df_t2'):
            tempDf = df_t2[df_t2['mnis_id']==mpId] 

        if (tempDf.shape[0]==0):
            continue
            
        tempList.extend(tempDf['Lemmas'].to_list())
        party = tempDf['party'].iat[0]
    
        #Flatten the list so it's not a list of lists
        tempList = [item for sublist in tempList for item in sublist]
        
        tempDf = pd.DataFrame([[mpId, party, tempList]],columns=['mnis_id', 'party', 'Lemmas'])
        dictSpeechesByMp[dfName]= tempDf
        dictSpeechesByMp[dfName]['df_name'] = dfName


In [13]:
len(dictSpeechesByMp.keys())

In [14]:
'''# Do not run, for verification only

len(dictSpeechesByMp.keys()) 
# Result 1302 

df_t1['mnis_id'].value_counts()
df_t2['mnis_id'].value_counts()
# Result is 651 each - hence all covered in our dictionary DFs '''

In [15]:
%%time 
mpTimeDf = pd.DataFrame(columns = ['mnis_id', 'party', 'Lemmas', 'df_name'])
for val in list(dictSpeechesByMp.values()):
    mpTimeDf = mpTimeDf.append(val)

mpTimeDf['LengthLemmas'] = mpTimeDf.Lemmas.map(len)
mpTimeDf.agg(Max=('LengthLemmas', max), Min=('LengthLemmas', 'min'), Mean=('LengthLemmas', np.mean))

In [16]:
mpTimeDf = mpTimeDf[mpTimeDf['LengthLemmas']>60000] 
mpTimeDf.shape

In [17]:
'''# Plotting vocab per MP per timeperiod distribution

plt.hist(vocabDf['LengthLemmas'], bins = 50)
plt.show()'''

In [18]:
# Checking for cutoff for vocab size
# Choosing all vocabs which have more than 10000 words

#vocabDf = vocabDf[vocabDf['LengthLemmas']>6000] 
#- 921 rows

#vocabDf[vocabDf['LengthLemmas']>1000] = 1287 rows
#vocabDf[vocabDf['LengthLemmas']>5000] - 1123 rows


In [19]:
change = ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender',
          'departure', 'triggering', 'stockpiling', 'expulsion', 'blindfold', 'cliff', 'lighter', 'exits', 'triggered',
          'brexiteer', 'soft', 'plus', 'trigger', 'backroom', 'invoked', 'protesting', 'brexit', 'edge', 'canary', 
          'unicorns', 'withdrawing', 'invoking', 'withdrawn', 'manor', 'brexiteers', 'fanatics', 'postponement', 
          'currencies', 'currency', 'operability', 'operable', 'leavers', 'invoke', 'article', 'eurozone', 'clueless',
          'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'lining', 'gove', 'norway', 'deflationary',
          'moribund', 'eurosceptic', 'deutschmark', 'courting', 'deal', 'withdraw', 'dab', 'withdrawals', 'eurosceptics',
          'surrendering', 'aldous', 'lanarkshire', 'leaving', 'signifying', 'roofs', 'ceded', 'absentia', 'treachery',
          'dollar', 'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum',
          'agreement', 'prorogue', 'smoothest', 'depreciate', 'managed', 'mutiny', 'overvalued', 'ideologues', 'foreign',
          'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 'southbound', 'left', 'line', 'hard', 'bill']
 

In [20]:
mpTimeDf = mpTimeDf.reset_index()

In [21]:
# mpTimeDf['overlap2_delist'] = str(mpTimeDf['overlapping_words2'])
# *** **** **** Forceful filtering to keep brexit and leave words **** **** **** 
mpTimeDf['lemmas_delist'] = [','.join(map(str, l)) for l in mpTimeDf['Lemmas']]
mpTimeDf = mpTimeDf[mpTimeDf['lemmas_delist'].str.contains(r'\bbrexit\b')]
mpTimeDf = mpTimeDf[mpTimeDf['lemmas_delist'].str.contains(r'\bleave\b')]
#mpTimeDf = mpTimeDf[mpTimeDf['lemmas_delist'].str.contains(r'\bleave\b')]

mpTimeDf.shape

In [22]:
mpTimeDf

In [23]:
#mpTimeDf = mpTimeDf[mpTimeDf['lemmas_delist'].str.contains('departure')]
#mpTimeDf.shape

In [24]:
%%time
overlappingWords = []
for ind in mpTimeDf.index:
    mpVocabInTime = mpTimeDf.at[ind, 'Lemmas']
    #print(mpVocabInTime)
    overlap = list(set(mpVocabInTime).intersection(change))
    overlappingWords.append(overlap)

mpTimeDf['overlapping_words'] = overlappingWords
mpTimeDf['overlapCount'] = mpTimeDf.overlapping_words.map(len)

In [25]:
mpTimeDf.shape

In [26]:
# Now reducing our changed words to include these AND also words that we DEFINITELY WANT

listOfOverlapLists = mpTimeDf['overlapping_words'].to_list()

# ******* CONFIRM WHY [0] ******

x = set(listOfOverlapLists[0]).intersection(*listOfOverlapLists)

#set([1])
#x.add('brexit') #58 from 132
#x.add('leave') #58 from 132

#x.add('remain')
#x.add('exit') #60now
#x.add('cliff')
#x.add('trigger')
#x.add('withdraw')
#x.add('seaborne') #40 now
#x.add('remainers') #now 5 or 7 

#x = list(x).append('brexit')

# When vocab above 6000 considered, 'deal' is common
# When vocab above 20,000 considered, common vocab is {'deal', 'hard', 'leave', 'left'}
# Vocab above 40,000 considered , common words:
#{'agreement', 'bill', 'control', 'deal', 'hard', 'leave', 'left', 'line'}
# Vocab above 60,000, vocab - 
#{'agreement', 'bill', 'control', 'deal', 'hard', 'leave', 'left', 'line'}

 
x=list(x)
x

In [27]:
'''overlappingWords2 = []
for ind in mpTimeDf.index:
    mpVocabInTime = mpTimeDf.at[ind, 'Lemmas']
    #print(mpVocabInTime)
    overlap = list(set(mpVocabInTime).intersection(x))
    overlappingWords2.append(overlap)

mpTimeDf['overlapping_words2'] = overlappingWords2
mpTimeDf['overlapCount2'] = mpTimeDf.overlapping_words2.map(len)
mpTimeDf['overlapCount2'].describe()'''

In [28]:
#mpTimeDf['overlapping_words2'].value_counts()

In [29]:
'''#mpTimeDf['overlap2_delist'] = str(mpTimeDf['overlapping_words2'])
mpTimeDf['overlap2_delist'] = [','.join(map(str, l)) for l in mpTimeDf['overlapping_words2']]
mpTimeDf_overidden = mpTimeDf[mpTimeDf['overlap2_delist'].str.contains('brexit')]
mpTimeDf_overidden = mpTimeDf[mpTimeDf['overlap2_delist'].str.contains('leave')]'''

In [30]:
mpTimeDf.shape

In [31]:
mpTimeDf

In [32]:
'''# We should only keep speeches by those MPs which have occurences in both T1 & T2 since comparison in their vocab is to be made here

#list(vocabDf['mnis_id'].value_counts().value_counts())
#340 MPs have speeches in both T1 and T, 241 have speech(es) in one of the time periods

mpCountDict = vocabDf['mnis_id'].value_counts().to_dict()

vocabDf['mpCountInTimes'] = vocabDf['mnis_id'].map(mpCountDict)

#Drop the MPs with selected vocab in only one time interval

vocabDf=vocabDf[vocabDf['mpCountInTimes']==2]

# For manual verification'''
'''vocabDf[vocabDf['mnis_id']=='3942']
3992, 3942 - only one instance of each
4263 - 2 instances'''

In [33]:
'''# Now backtrack and update dictionary to only keep DFs of these 921 rows

# Creating dict key names to keep from our filtered DF mnis IDs

mnis_id_list = list(vocabDf['mnis_id'].value_counts().to_dict().keys())

list1 = ['df_t1_'+item for item in mnis_id_list]
list2 = ['df_t2_'+item for item in mnis_id_list]
mnis_id_list = list1 + list2'''

In [34]:
'''mnis_id_list = list(vocabDf['mnis_id'].value_counts().to_dict().keys())
len(mnis_id_list)'''

In [35]:
listDfsKeep = mpTimeDf['df_name'].to_list()

In [36]:
len(listDfsKeep)

In [37]:
# Dropping key-value pairs from dictionary where the key doesn't match

for k,v in list(dictSpeechesByMp.items()):
    if (k not in listDfsKeep):
        del dictSpeechesByMp[k]

In [38]:
len(dictSpeechesByMp.values())

In [39]:
%%time

#  - x - x - x - CREATING & SAVING WORD2VEC MODELS FOR THE 680 MODELS - x - x - x 

dictOfModels = {}
'''
import shutil
shutil.rmtree('./models-by-mp-and-time')
'''
os.makedirs('./models-by-mp-and-time')
models_folder = './models-by-mp-and-time'
count = 1

for dframe in dictSpeechesByMp: 
    
# Doing in batches since notebook RAM crashe
    party = dictSpeechesByMp[dframe]['party'].values[0]
    if('&' in party):
        #print('............',party.split('& '))
        splitParty = party.split('&')
        party = ''
        for phr in splitParty:
            #print('dddd')
            party = party + str(phr)
        #print('11111111111111111',party)
    model = gensim.models.Word2Vec(dictSpeechesByMp[dframe]['Lemmas'], min_count=1, vector_size=300, window = 5, sg = 1)

    # Also saving model in a dict and exporting

    modelName ='model_'+ dframe+'_'+party
    print('model number', count, modelName)

    dictOfModels[dframe] = model
    model.save(os.path.join(models_folder, modelName))
    count = count +1
    


In [40]:
'''%%time
# Commenting since we're going to load and start with alignment of models with a fresh notebook
functools.reduce(smart_procrustes_align_gensim, list(dictOfModels.values()))'''

In [41]:
%%time
!zip -r file.zip /kaggle/working/models-by-mp-and-time

In [ ]:
# To delete if need be
'''import shutil
shutil.rmtree('./models-by-mp-and-time')

os.remove('./file.zip')'''

In [ ]:
dictOfModels

In [ ]:
%%time
modelsToAlign = list(dictOfModels.values())
for i in range(0,len(modelsToAlign)-1):
    functools.reduce(smart_procrustes_align_gensim, modelsToAlign)

In [ ]:
dictOfModels.keys()
dictOfModels['df_t2_1423'].wv.index_to_key

In [ ]:
dictOfModels['df_t2_1423'].wv.vectors.shape

In [ ]:
t = 'brexit'
#dictOfModels['df_t2_1423'].wv.index_to_key
#dictOfModels['df_t2_1423'].layer1_size

kis = ['df_t2_1423','df_t2_3945', 'df_t2_4070']
modelsSum = np.zeros(dictOfModels['df_t2_1423'].layer1_size)

for k in kis:
    #Something like this to refer to t's vector
    #vector = dictOfModels[k].wv.get_index(t)#
    vector = dictOfModels[k].wv[t]
    modelsSum = np.add(modelsSum, vector)
    
avgEmbedding =np.divide(modelsSum, len(kis))


In [ ]:
avgEmbedding.shape

In [ ]:
dictOfModels['df_t2_1423'].wv.get_index('brexit')

In [ ]:
dictOfModels['df_t2_1423'].wv.get_item('brexit')

In [ ]:
def cosine_similarity(word):
  sc = 1-spatial.distance.cosine(dictOfModels['df_t2_3945'].wv[t], dictOfModels['df_t2_1423'].wv[t])
  return sc

In [ ]:
cosine_similarity('brexit')

In [ ]:
dictOfModels['df_t2_1423'].wv['brexit']

In [ ]:
avgEmbedding.shape

In [ ]:

'''def pattern2vector(tokens, word2vec, AVG=False):
    pattern_vector = np.zeros(word2vec.layer1_size)
    n_words = 0
    if len(tokens) > 1:
        for t in tokens:
            try:
                vector = word2vec[t.strip()]
                pattern_vector = np.add(pattern_vector,vector)
                n_words += 1
            except KeyError, e:
                continue
        if AVG is True:
            pattern_vector = np.divide(pattern_vector,n_words)
    elif len(tokens) == 1:
        try:
            pattern_vector = word2vec[tokens[0].strip()]
        except KeyError:
            pass
    return pattern_vector
'''


In [ ]:
#np.mean(dictOfModels['df_t2_1423'][['brexit']],axis=0)

In [ ]:
'df_t2_3945', 'df_t2_4070'
def pattern2vector(tokens, word2vec):
    pattern_vector = np.zeros(word2vec.layer1_size)
    n_words = 0
    if len(tokens) > 1:
        for t in tokens:
            try:
                vector = word2vec[t.strip()]
                pattern_vector = np.add(pattern_vector,vector)
                n_words += 1
            except KeyError, e:
                continue
        if AVG is True:
            pattern_vector = np.divide(pattern_vector,n_words)
    elif len(tokens) == 1:
        try:
            pattern_vector = word2vec[tokens[0].strip()]
        except KeyError:
            pass
    return pattern_vector

In [ ]:
def get_mean_vector(word2vec_models):
    # remove out-of-vocabulary words
    #words = [word for word in words if word in word2vec_model.vocab]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []


In [ ]:
dictOfModels
for i in dictOfModels.keys():
  print(dictOfModels[i], dictOfModels[i].wv.similar_by_word('brexit', 10))
  print('- x - x - x - x - x - x - x - x - x')

In [ ]:
dictOfModels.keys()

In [ ]:
set(dictOfModels['df_t1_1427'].wv.index_to_key).intersection(x)
# Common vocab of 1071, 10 common words with our words

In [ ]:
def cosine_similarity(word):
  sc = 1-spatial.distance.cosine(model1.wv[word], model2.wv[word])
  return sc

In [ ]:
%%time
cosine_similarity_df = pd.DataFrame(([w, cosine_similarity(w), model1.wv.get_vecattr(w, "count") , model2.wv.get_vecattr(w, "count") ] for w in model1.wv.index_to_key), columns = ('Word', 'Cosine_similarity', "Frequency_t1", "Frequency_t2"))

cosine_similarity_df['FrequencyRatio'] = cosine_similarity_df['Frequency_t1']/cosine_similarity_df['Frequency_t2']
cosine_similarity_df['TotalFrequency'] = cosine_similarity_df['Frequency_t1'] + cosine_similarity_df['Frequency_t2']

cosine_similarity_df_sorted = cosine_similarity_df.sort_values(by='Cosine_similarity', ascending=True)